<a href="https://colab.research.google.com/github/ferreirabruno7/Text_analyze/blob/master/previs%C3%A3o_cluster_texto_covid_19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#acessar google drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import nltk
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score

In [3]:
comentarios = pd.read_csv("/content/drive/My Drive/Colab Notebooks/covid_19_cluuster_new.csv", delimiter = ',', encoding='utf-8')
comentarios.head()

,Unnamed: 0,protocolo,data_manifestacao,municipio_denunciante,municipio_fato,localFato,manifestacao,status_atual,desc_status,sigla,nome,tipo_manifestacao,assunto,sub_assunto,tokens,cluster,coseno
0,0,2020.0314.215152-35,2020-03-14 21:51:52,NaN,GOIANIA,Setor Candida de Morais,Elogio com sugestão de ações,55,Finalizado pela ouvidoria do Órgão.,SGG/GOVERNADORIA,Secretaria Geral da Governadoria,Elogio,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...","['Elogio', 'com', 'sugestão', 'de', 'ações']",3,0.00000
1,1,2020.0316.203054-88,2020-03-16 20:30:54,NaN,GOIANIA,Vila Redenção,Na empresa BTCC (prestadora de Serviços para O...,55,Finalizado pela ouvidoria do Órgão.,SES,Secretaria de Estado da Saúde,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...","['Na', 'empresa', 'BTCC', '(', 'prestadora', '...",5,0.00000
2,2,2020.0318.081504-41,2020-03-18 08:15:04,NaN,ANAPOLIS,Vapt Vupt,Considerando o decreto do Sr. Governador quant...,55,Finalizado pela ouvidoria do Órgão.,SEAD,Secretaria de Estado de Administração (Antiga ...,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...","['Considerando', 'o', 'decreto', 'do', 'Sr', '...",5,0.03495
3,3,2020.0318.115152-35,2020-03-18 11:51:52,NaN,GOIANIA,POLÍCIA CÍVIL,Em meio a grave pandemia de coronavirus pela q...,55,Finalizado pela ouvidoria do Órgão.,SSP,Sec. Estado da Segurança Pública (Policias Civ...,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...","['Em', 'meio', 'a', 'grave', 'pandemia', 'de',...",5,0.00000
4,4,2020.0318.115845-69,2020-03-18 11:58:45,NaN,GOIANIA,NaN,Em virtude do Decreto emitido pelo Governador ...,55,Finalizado pela ouvidoria do Órgão.,SSP,Sec. Estado da Segurança Pública (Policias Civ...,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...","['Em', 'virtude', 'do', 'Decreto', 'emitido', ...",5,0.00000


In [4]:
comentarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             2129 non-null   int64  
 1   protocolo              2129 non-null   object 
 2   data_manifestacao      2129 non-null   object 
 3   municipio_denunciante  277 non-null    object 
 4   municipio_fato         2129 non-null   object 
 5   localFato              1841 non-null   object 
 6   manifestacao           2129 non-null   object 
 7   status_atual           2129 non-null   int64  
 8   desc_status            340 non-null    object 
 9   sigla                  2129 non-null   object 
 10  nome                   2129 non-null   object 
 11  tipo_manifestacao      2129 non-null   object 
 12  assunto                2129 non-null   object 
 13  sub_assunto            2129 non-null   object 
 14  tokens                 2129 non-null   object 
 15  clus

In [0]:
del comentarios['Unnamed: 0']
del comentarios['tokens']
del comentarios['coseno']


In [6]:
comentarios.describe()

,status_atual,cluster
count,2129.000000,2129.000000
mean,5.827619,3.833255
std,12.599561,1.671832
min,1.000000,0.000000
25%,1.000000,3.000000
50%,1.000000,4.000000
75%,1.000000,5.000000
max,55.000000,7.000000


In [7]:
comentarios['cluster'].value_counts()

3    552
5    509
2    370
6    216
4    203
7    141
1    125
0     13
Name: cluster, dtype: int64

In [8]:
comentarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2129 entries, 0 to 2128
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   protocolo              2129 non-null   object
 1   data_manifestacao      2129 non-null   object
 2   municipio_denunciante  277 non-null    object
 3   municipio_fato         2129 non-null   object
 4   localFato              1841 non-null   object
 5   manifestacao           2129 non-null   object
 6   status_atual           2129 non-null   int64 
 7   desc_status            340 non-null    object
 8   sigla                  2129 non-null   object
 9   nome                   2129 non-null   object
 10  tipo_manifestacao      2129 non-null   object
 11  assunto                2129 non-null   object
 12  sub_assunto            2129 non-null   object
 13  cluster                2129 non-null   int64 
dtypes: int64(2), object(12)
memory usage: 233.0+ KB


In [0]:
def pre_process(texto):
  texto = texto.replace(' nan ','')
  texto = texto.replace('nan ','')
  texto = texto.replace(' nan','')
  texto = re.sub('\r','',texto)
  texto = re.sub('\n','',texto)
  texto = re.sub(',',' ',texto)
  texto = re.sub('\.',' ',texto)
  texto = re.sub('\?',' ',texto)
  texto = re.sub('\!',' ',texto)
  texto = re.sub('\;',' ',texto)
  return texto

In [0]:
comentarios['new_manifestacao'] = comentarios.manifestacao.apply(pre_process)

In [0]:
texto = comentarios['new_manifestacao'].values
classes = comentarios['cluster'].values

In [12]:
## Definido o corpus

corpus = comentarios.new_manifestacao.values
corpus

array(['Elogio com sugestão de ações ',
       'Na empresa BTCC (prestadora de Serviços para OI) não esta havendo nenhuma ação (alteração/prevenção) visando evitar a possível propagação do CORONAVIRUS entre seus colaboradores (funcionários)  haja visto que a empresa possui quase 10 mil funcionários em plena atividade em Goiânia  cujo contato e aglomeração favorecem esse tipo de disseminação tão combatida nesse momento de crise Este fato se estende as demais Empresas Operadores de Telemarketing que  mesmo possuindo quantidades menores de funcionários  se tornam possíveis focos de propagação do Covid-19  ',
       'Considerando o decreto do Sr  Governador quanto às medidas de enfrentamento ao Coronavírus  quero fazer minha reclamação acerca dos Vaptvutps:Estão funcionando normalmente e pretendem funcionar a despeito do decreto  inclusive em Shopping Centers Os gestores alegam que os atendimentos estão sendo por agendamento  e realmente estão sendo assim  Porém  a cada 30 minutos são disp

In [13]:
## Vetorizando o corpus (e criando nosso X)

cvec = CountVectorizer(ngram_range=(2,2),min_df=1)
x = cvec.fit_transform(corpus)
x

<2129x42930 sparse matrix of type '<class 'numpy.int64'>'
	with 79920 stored elements in Compressed Sparse Row format>

In [14]:
cvec.vocabulary_

{'elogio com': 13778,
 'com sugestão': 7653,
 'sugestão de': 38251,
 'de ações': 10077,
 'na empresa': 26260,
 'empresa btcc': 14352,
 'btcc prestadora': 5470,
 'prestadora de': 32656,
 'de serviços': 10924,
 'serviços para': 37435,
 'para oi': 29825,
 'oi não': 28644,
 'não esta': 27966,
 'esta havendo': 15746,
 'havendo nenhuma': 20925,
 'nenhuma ação': 26812,
 'ação alteração': 4784,
 'alteração prevenção': 2706,
 'prevenção visando': 32704,
 'visando evitar': 42252,
 'evitar possível': 16774,
 'possível propagação': 32205,
 'propagação do': 33000,
 'do coronavirus': 12805,
 'coronavirus entre': 9047,
 'entre seus': 15118,
 'seus colaboradores': 37594,
 'colaboradores funcionários': 7112,
 'funcionários haja': 19544,
 'haja visto': 20907,
 'visto que': 42297,
 'que empresa': 33992,
 'empresa possui': 14617,
 'possui quase': 32183,
 'quase 10': 33800,
 '10 mil': 191,
 'mil funcionários': 25378,
 'funcionários em': 19484,
 'em plena': 14047,
 'plena atividade': 31262,
 'atividade em':

In [15]:
## Definindo Y

y = np.array(comentarios['cluster'].values)
y

array([3, 5, 5, ..., 5, 2, 2])

In [0]:
vectorizer = CountVectorizer()
freq_text = vectorizer.fit(texto)


In [0]:
freq_text = vectorizer.transform(texto)


Modelo Multinomial

In [20]:
modelo = MultinomialNB()
modelo.fit(freq_text,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
testes = ['Temos que fechar as indústrias, somos todos vetores de uma possível transmissão']

In [23]:
freq_testes = vectorizer.transform(testes)
modelo.predict(freq_testes)

array([5])

In [0]:
resultados = cross_val_predict(modelo, freq_text, classes, cv=10)


In [25]:
metrics.accuracy_score(classes,resultados)


0.565054015969939

In [26]:
print(classification_report(resultados, classes))

              precision    recall  f1-score   support

           0       0.69      1.00      0.82         9
           1       0.13      0.89      0.22        18
           2       0.61      0.58      0.59       389
           3       0.56      0.67      0.61       456
           4       0.26      0.79      0.39        67
           5       0.98      0.46      0.63      1079
           6       0.32      0.80      0.46        86
           7       0.18      1.00      0.30        25

    accuracy                           0.57      2129
   macro avg       0.47      0.77      0.50      2129
weighted avg       0.75      0.57      0.60      2129



Modelo Gaussian

In [0]:
from sklearn.naive_bayes import GaussianNB
classificador = GaussianNB()

In [0]:
X = freq_text.todense()


In [0]:
classificador.fit(X,y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [0]:
testes2 = ['Temos que fechar as indústrias, somos todos vetores de uma possível transmissão']
freq_testes2 = vectorizer.transform(testes2)


In [0]:
X_teste = freq_testes2.todense()


In [0]:
freq_testes_gaussian = vectorizer.transform(testes2)
classificador.predict(X_teste)

array([5])

In [0]:
resultados = cross_val_predict(classificador, X, classes, cv=10)


In [0]:
metrics.accuracy_score(classes,resultados)


0.3771723813997182

In [0]:
print(classification_report(resultados, classes))

              precision    recall  f1-score   support

           0       0.85      1.00      0.92        11
           1       0.21      0.27      0.23        98
           2       0.25      0.33      0.28       278
           3       0.31      0.41      0.35       422
           4       0.15      0.29      0.20       108
           5       0.85      0.41      0.55      1058
           6       0.13      0.27      0.18       104
           7       0.08      0.22      0.12        50

    accuracy                           0.38      2129
   macro avg       0.35      0.40      0.35      2129
weighted avg       0.55      0.38      0.42      2129



Arvore de Decisão

In [0]:
from sklearn.tree import DecisionTreeClassifier

In [0]:
dtree = DecisionTreeClassifier()

In [0]:
dtree.fit(X,y)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [0]:
testes3 = ['Temos que fechar as indústrias, somos todos vetores de uma possível transmissão']
freq_testes3 = vectorizer.transform(testes3)


In [0]:
dtree.predict(freq_testes3)

array([3])

In [0]:
resultados = cross_val_predict(dtree, X, classes, cv=10)


In [0]:
metrics.accuracy_score(classes,resultados)


0.6679192108971348

In [0]:
print(classification_report(resultados, classes))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        13
           1       0.61      0.62      0.61       123
           2       0.57      0.60      0.59       351
           3       0.66      0.68      0.67       537
           4       0.67      0.64      0.66       211
           5       0.74      0.71      0.72       527
           6       0.63      0.65      0.64       211
           7       0.76      0.69      0.72       156

    accuracy                           0.67      2129
   macro avg       0.71      0.70      0.70      2129
weighted avg       0.67      0.67      0.67      2129



Floresta Aleatória

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
rcf = RandomForestClassifier(n_estimators=500)

In [0]:
rcf.fit(X,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [0]:
testes4 = ['Temos que fechar as indústrias, somos todos vetores de uma possível transmissão']
freq_testes4 = vectorizer.transform(testes4)

In [0]:
rcf.predict(freq_testes4)

array([3])

In [0]:
resultados = cross_val_predict(rcf, X, classes, cv=10)


In [0]:
metrics.accuracy_score(classes,resultados)


0.7844058243306716

In [0]:
print(classification_report(resultados, classes))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96        12
           1       0.51      0.91      0.66        70
           2       0.82      0.72      0.77       417
           3       0.77      0.79      0.78       540
           4       0.76      0.85      0.81       182
           5       0.90      0.75      0.82       607
           6       0.72      0.85      0.78       182
           7       0.70      0.83      0.76       119

    accuracy                           0.78      2129
   macro avg       0.76      0.84      0.79      2129
weighted avg       0.80      0.78      0.79      2129



In [27]:
comentarios_teste = pd.read_csv("/content/drive/My Drive/Colab Notebooks/manifestacao_10.csv", delimiter = ';', encoding='iso-8859-1')
comentarios_teste.head()

,protocolo,data_manifestacao,municipio_denunciante,municipio_fato,localFato,manifestacao,status_atual,desc_status,sigla,nome,tipo_manifestacao,assunto,sub_assunto
0,2020.0314.215152-35,2020-03-14 21:51:52,NaN,GOIANIA,Setor Candida de Morais,Elogio com sugestão de ações,55,Finalizado pela ouvidoria do Órgão.,SGG/GOVERNADORIA,Secretaria Geral da Governadoria,Elogio,Saúde,"Corona Vírus - Denúncia, solicitação ou inform..."
1,2020.0316.083654-7,2020-03-16 08:36:54,NaN,FORMOSA,Case,Escolas fechadasEscritórios fechadosShopping v...,55,Finalizado pela ouvidoria do Órgão.,SEDS,Secretaria de Estado de Desenvolvimento Social...,Comunicação,COVID-19 - Corona Vírus,Estabelecimento aberto
2,2020.0316.115913-32,2020-03-16 11:59:13,NaN,GOIANIA,Seduc,Trabalho na Seduc a muitos anos. Hoje viemos t...,55,Finalizado pela ouvidoria do Órgão.,SEDUC,Secretaria de Estado da Educação,Comunicação,COVID-19 - Corona Vírus,Outros
3,2020.0316.203054-88,2020-03-16 20:30:54,NaN,GOIANIA,Vila Redenção,Na empresa BTCC (prestadora de Serviços para O...,55,Finalizado pela ouvidoria do Órgão.,SES,Secretaria de Estado da Saúde,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform..."
4,2020.0317.091024-64,2020-03-17 09:10:24,NaN,GOIANIA,Via pública,"O reclamante, Sr. Marcos Vinícius de Araújo, d...",55,Finalizado pela ouvidoria do Órgão.,SSP,Sec. Estado da Segurança Pública (Policias Civ...,Reclamação,COVID-19 - Corona Vírus,Estabelecimento aberto


In [28]:
comentarios_teste.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   protocolo              3168 non-null   object
 1   data_manifestacao      3168 non-null   object
 2   municipio_denunciante  421 non-null    object
 3   municipio_fato         3168 non-null   object
 4   localFato              2732 non-null   object
 5   manifestacao           3168 non-null   object
 6   status_atual           3168 non-null   int64 
 7   desc_status            2218 non-null   object
 8   sigla                  3168 non-null   object
 9   nome                   3168 non-null   object
 10  tipo_manifestacao      3168 non-null   object
 11  assunto                3168 non-null   object
 12  sub_assunto            3168 non-null   object
dtypes: int64(1), object(12)
memory usage: 321.9+ KB


In [0]:
comentarios_teste['new_manifestacao'] = comentarios_teste.manifestacao.apply(pre_process)

In [0]:
texto_2 = comentarios_teste['new_manifestacao'].values


In [31]:
corpus_2 = comentarios_teste.new_manifestacao.values
corpus_2

array(['Elogio com sugestão de ações ',
       'Escolas fechadasEscritórios fechadosShopping vaziosEventos desmarcadosFérias canceladasVoos canceladosEconomia recuandoAté mesmo fronteiras internacionais trancadasPopulação assustada  olhando umas as outras e esquivando de contato     e o CASE de Formosa  sem gestão  continua funcionando NORMALMENTE ',
       'Trabalho na Seduc a muitos anos  Hoje viemos trabalhar com a expectativa de respeitarem o Decreto do governo sobre o corona vírus  Mas para nossa surpresa  aqui não estão nem aí Trabalhamos no dia a dia com banheiros sujos   só existe papel e melhores condições no banheiro do pavilhão de cima  Pq as pessoas do gabinete frequentam lá  Aqui embaixo é o caos Pensamos que teríamos o álcool gel   as toalhas de papel   o mínimo de condições previstas no decreto  ao contrário   ninguém tá nem ai  Trabalhamos atendendo pessoas que vem do estado inteiro  Não nos respeitam como servidores   não respeitam nossas famílias    Essa pandemia    a

In [32]:
corpus_2.reshape(1, -1)

array([['Elogio com sugestão de ações ',
        'Escolas fechadasEscritórios fechadosShopping vaziosEventos desmarcadosFérias canceladasVoos canceladosEconomia recuandoAté mesmo fronteiras internacionais trancadasPopulação assustada  olhando umas as outras e esquivando de contato     e o CASE de Formosa  sem gestão  continua funcionando NORMALMENTE ',
        'Trabalho na Seduc a muitos anos  Hoje viemos trabalhar com a expectativa de respeitarem o Decreto do governo sobre o corona vírus  Mas para nossa surpresa  aqui não estão nem aí Trabalhamos no dia a dia com banheiros sujos   só existe papel e melhores condições no banheiro do pavilhão de cima  Pq as pessoas do gabinete frequentam lá  Aqui embaixo é o caos Pensamos que teríamos o álcool gel   as toalhas de papel   o mínimo de condições previstas no decreto  ao contrário   ninguém tá nem ai  Trabalhamos atendendo pessoas que vem do estado inteiro  Não nos respeitam como servidores   não respeitam nossas famílias    Essa pandemia  

In [34]:
cvec_teste = CountVectorizer(vocabulary=vectorizer.vocabulary)
x_teste = cvec_teste.fit_transform(corpus_2)
x_teste.reshape(1, -1)

<1x38234592 sparse matrix of type '<class 'numpy.int64'>'
	with 114145 stored elements in COOrdinate format>

In [35]:
cvec_teste.vocabulary_

{'elogio': 4598,
 'com': 2961,
 'sugestão': 10826,
 'de': 3765,
 'ações': 1998,
 'escolas': 4911,
 'fechadasescritórios': 5422,
 'fechadosshopping': 5426,
 'vazioseventos': 11628,
 'desmarcadosférias': 4049,
 'canceladasvoos': 2465,
 'canceladoseconomia': 2467,
 'recuandoaté': 9836,
 'mesmo': 7656,
 'fronteiras': 5762,
 'internacionais': 6671,
 'trancadaspopulação': 11301,
 'assustada': 1774,
 'olhando': 8309,
 'umas': 11468,
 'as': 1720,
 'outras': 8423,
 'esquivando': 5014,
 'contato': 3351,
 'case': 2578,
 'formosa': 5679,
 'sem': 10479,
 'gestão': 5949,
 'continua': 3368,
 'funcionando': 5790,
 'normalmente': 8116,
 'trabalho': 11284,
 'na': 7977,
 'seduc': 10435,
 'muitos': 7889,
 'anos': 1531,
 'hoje': 6219,
 'viemos': 11782,
 'trabalhar': 11274,
 'expectativa': 5236,
 'respeitarem': 10069,
 'decreto': 3797,
 'do': 4375,
 'governo': 6048,
 'sobre': 10681,
 'corona': 3497,
 'vírus': 11955,
 'mas': 7515,
 'para': 8510,
 'nossa': 8127,
 'surpresa': 10882,
 'aqui': 1657,
 'não': 8190

In [36]:
cvec_teste

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [0]:
vectorizer_2 = CountVectorizer(analyzer="word")
freq_text_ = vectorizer.transform(texto_2)

In [46]:
pred = modelo.predict_proba(freq_text_)
pred

array([[2.45122431e-004, 1.00973537e-002, 9.13481992e-002, ...,
        2.90455837e-002, 4.18176894e-002, 2.33388166e-002],
       [2.03345300e-011, 1.43550000e-004, 4.67974251e-001, ...,
        2.06396930e-001, 9.13831069e-002, 6.09350780e-004],
       [5.45238033e-126, 5.45165424e-056, 1.59511120e-024, ...,
        1.00000000e+000, 3.00260849e-033, 1.74176669e-051],
       ...,
       [4.04629723e-067, 4.58972278e-036, 3.54600278e-015, ...,
        1.00000000e+000, 1.24723908e-025, 2.00778256e-028],
       [8.18843207e-008, 4.68266349e-005, 9.32076791e-001, ...,
        1.73020053e-002, 2.91023933e-002, 1.77519954e-004],
       [6.58478959e-017, 6.11872034e-009, 1.30290281e-001, ...,
        4.59538503e-003, 5.32869405e-005, 2.35841698e-004]])

In [0]:
previsageral = pd.DataFrame(pred)

In [40]:
previsageral.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3168 entries, 0 to 3167
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       3168 non-null   float64
 1   1       3168 non-null   float64
 2   2       3168 non-null   float64
 3   3       3168 non-null   float64
 4   4       3168 non-null   float64
 5   5       3168 non-null   float64
 6   6       3168 non-null   float64
 7   7       3168 non-null   float64
dtypes: float64(8)
memory usage: 198.1 KB


In [41]:
previsageral.round(3)

,0,1,2,3,4,5,6,7
0,0.0,0.01,0.091,0.632,0.172,0.029,0.042,0.023
1,0.0,0.00,0.468,0.176,0.058,0.206,0.091,0.001
2,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
3,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
4,0.0,0.00,0.000,0.999,0.000,0.001,0.000,0.000
...,...,...,...,...,...,...,...,...
3163,0.0,0.00,0.000,0.995,0.000,0.005,0.000,0.000
3164,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
3165,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
3166,0.0,0.00,0.932,0.019,0.002,0.017,0.029,0.000


In [42]:
previsageral.round(3)

,0,1,2,3,4,5,6,7
0,0.0,0.01,0.091,0.632,0.172,0.029,0.042,0.023
1,0.0,0.00,0.468,0.176,0.058,0.206,0.091,0.001
2,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
3,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
4,0.0,0.00,0.000,0.999,0.000,0.001,0.000,0.000
...,...,...,...,...,...,...,...,...
3163,0.0,0.00,0.000,0.995,0.000,0.005,0.000,0.000
3164,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
3165,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
3166,0.0,0.00,0.932,0.019,0.002,0.017,0.029,0.000


In [43]:
previsageral

,0,1,2,3,4,5,6,7
0,2.451224e-04,1.009735e-02,9.134820e-02,6.319433e-01,1.721640e-01,0.029046,4.181769e-02,2.333882e-02
1,2.033453e-11,1.435500e-04,4.679743e-01,1.758201e-01,5.767267e-02,0.206397,9.138311e-02,6.093508e-04
2,5.452380e-126,5.451654e-56,1.595111e-24,5.454201e-33,1.850688e-44,1.000000,3.002608e-33,1.741767e-51
3,3.732885e-61,5.965408e-28,1.430745e-13,2.238622e-20,3.328012e-25,1.000000,7.468892e-19,1.077054e-31
4,6.931288e-98,3.241594e-36,9.830190e-26,9.986871e-01,1.124510e-28,0.001313,1.342158e-14,2.054176e-28
...,...,...,...,...,...,...,...,...
3163,5.455170e-19,1.282164e-12,7.213144e-07,9.949161e-01,5.964687e-10,0.005082,1.284267e-06,7.432425e-10
3164,5.383951e-97,2.042450e-40,7.087213e-15,1.144415e-20,4.721499e-32,1.000000,5.461918e-25,4.017955e-35
3165,4.046297e-67,4.589723e-36,3.546003e-15,7.718744e-16,9.397013e-28,1.000000,1.247239e-25,2.007783e-28
3166,8.188432e-08,4.682663e-05,9.320768e-01,1.896441e-02,2.329976e-03,0.017302,2.910239e-02,1.775200e-04


In [44]:
previsaofinal = pd.concat([comentarios_teste,previsageral.round(3)], axis=1)
previsaofinal.head(1000)

,protocolo,data_manifestacao,municipio_denunciante,municipio_fato,localFato,manifestacao,status_atual,desc_status,sigla,nome,tipo_manifestacao,assunto,sub_assunto,new_manifestacao,0,1,2,3,4,5,6,7
0,2020.0314.215152-35,2020-03-14 21:51:52,NaN,GOIANIA,Setor Candida de Morais,Elogio com sugestão de ações,55,Finalizado pela ouvidoria do Órgão.,SGG/GOVERNADORIA,Secretaria Geral da Governadoria,Elogio,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...",Elogio com sugestão de ações,0.0,0.01,0.091,0.632,0.172,0.029,0.042,0.023
1,2020.0316.083654-7,2020-03-16 08:36:54,NaN,FORMOSA,Case,Escolas fechadasEscritórios fechadosShopping v...,55,Finalizado pela ouvidoria do Órgão.,SEDS,Secretaria de Estado de Desenvolvimento Social...,Comunicação,COVID-19 - Corona Vírus,Estabelecimento aberto,Escolas fechadasEscritórios fechadosShopping v...,0.0,0.00,0.468,0.176,0.058,0.206,0.091,0.001
2,2020.0316.115913-32,2020-03-16 11:59:13,NaN,GOIANIA,Seduc,Trabalho na Seduc a muitos anos. Hoje viemos t...,55,Finalizado pela ouvidoria do Órgão.,SEDUC,Secretaria de Estado da Educação,Comunicação,COVID-19 - Corona Vírus,Outros,Trabalho na Seduc a muitos anos Hoje viemos t...,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
3,2020.0316.203054-88,2020-03-16 20:30:54,NaN,GOIANIA,Vila Redenção,Na empresa BTCC (prestadora de Serviços para O...,55,Finalizado pela ouvidoria do Órgão.,SES,Secretaria de Estado da Saúde,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...",Na empresa BTCC (prestadora de Serviços para O...,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
4,2020.0317.091024-64,2020-03-17 09:10:24,NaN,GOIANIA,Via pública,"O reclamante, Sr. Marcos Vinícius de Araújo, d...",55,Finalizado pela ouvidoria do Órgão.,SSP,Sec. Estado da Segurança Pública (Policias Civ...,Reclamação,COVID-19 - Corona Vírus,Estabelecimento aberto,O reclamante Sr Marcos Vinícius de Araújo d...,0.0,0.00,0.000,0.999,0.000,0.001,0.000,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,2020.0323.092019-48,2020-03-23 09:20:19,NaN,GOIANIA,Nome empresarial Agx Florestal Eireli / Títul...,Quero denunciar a empresa AGX Madeiras situada...,1,NaN,SSP,Sec. Estado da Segurança Pública (Policias Civ...,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...",Quero denunciar a empresa AGX Madeiras situada...,0.0,0.00,0.746,0.000,0.000,0.001,0.253,0.000
996,2020.0323.092035-33,2020-03-23 09:20:35,NaN,GOIANIA,Unimed centro clínico,"A unimed centro clinico,situado na rua 104 no ...",55,Finalizado pela ouvidoria do Órgão.,SES,Secretaria de Estado da Saúde,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...",A unimed centro clinico situado na rua 104 no ...,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
997,2020.0323.092719-49,2020-03-23 09:27:19,NaN,APARECIDA DE GOIANIA,"Rua Nilo bufaical, qd 42, antiga rua 2, vila M...",ESTA EMPRESA AINDA ESTA EM ABERTO E SEM QUALQU...,21,Sob análise do orgão responsável.,SSP,Sec. Estado da Segurança Pública (Policias Civ...,Solicitação,COVID-19 - Corona Vírus,Estabelecimento aberto,ESTA EMPRESA AINDA ESTA EM ABERTO E SEM QUALQU...,0.0,0.00,0.788,0.000,0.000,0.212,0.000,0.000
998,2020.0323.093453-24,2020-03-23 09:34:53,NaN,GOIANIA,Conjunto primavera Rua cp58 n437,Tem uma loja de internet situada na Rua cp58 n...,21,Sob análise do orgão responsável.,SSP,Sec. Estado da Segurança Pública (Policias Civ...,Comunicação,Saúde,"Corona Vírus - Denúncia, solicitação ou inform...",Tem uma loja de internet situada na Rua cp58 n...,0.0,0.00,0.000,0.000,0.000,1.000,0.000,0.000
